In [ ]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_openai import ChatOpenAI

In [ ]:
reader = "../test data/sample 1.pdf"
loader = UnstructuredPDFLoader(reader)

In [ ]:
data = loader.load()
for doc in data:
    data = doc.page_content

In [ ]:
import pytesseract
from PIL import Image
from pdf2image import convert_from_path
import os

# Convert PDF to images
doc_path = "../test data/sample 1.pdf"

_, ext = os.path.splitext(doc_path)

if ext == ".pdf":
    images = convert_from_path(doc_path)
    # Iterate over the images and extract text
    for i in range(len(images)):
        text = pytesseract.image_to_string(images[i], lang="eng")
        print(text)
else:
    print("file is not a pdf")

In [ ]:
from difflib import SequenceMatcher


def similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()

ground_truth = data

extracted_text = text

print(similarity(ground_truth, extracted_text))  ## 0.9637987776210625

In [ ]:
import nltk
import re


# Text cleaning
def clean_text(text):
    text = text.replace("\n", " ")  # replace newline characters with space
    text = re.sub(r"[^\w\s]", "", text)  # remove punctuation
    text = text.strip()  # remove leading and trailing whitespace
    return text


cleaned_text = clean_text(text=extracted_text)


In [ ]:
from pydantic.v1 import SecretStr

api_key = SecretStr("...")

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    api_key=api_key,
)

In [ ]:
messages = [
    (
        "system",
        "You are a helpful assistant that Identifying and extracting entities like names, dates, Address, email, Phone Number, etc. Extracting relationships between entities. Summarizing key information from the document.",
    ),
    ("human", cleaned_text),
]

In [ ]:
ai_msg = llm.invoke(messages)
ai_msg

In [ ]:
print(ai_msg.content)